In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Conv2D, Conv1D, MaxPooling2D, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow import keras, config
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print("GPUs Available: ", len(config.list_physical_devices('GPU')))

In [ ]:
def show_image(img, width, position = 0):
    plt.figure(figsize=(30,5))
    plt.xlim(0, width)
    if position != 0:
        plt.xticks(position)    
    plt.imshow(img)
    plt.show()

In [ ]:
# %load_ext autoreload
# %autoreload 2
from image_process.image_preprocessors import add_noise,add_noise_randomly, shift_random_update_positions
from data_filters import min_n_letters, take_percent
from keras_generators.image_generator import Image_Generator_RAW
# Global settings
TEST_SPLIT_SIZE = 0.80
VALIDATION_SPLIT_SIZE = 0.90
IMAGE_TARGET_SIZE = (5, 1400, 1)
BATCH_SIZE = 128
EPOCH_SIZE = 100
ADD_NOISE_RANDOMLY = [-2, 20]
ADD_SIGNAL_INDENT_RANDOMLY = [0, 1000]
MASKS = [
    {"func": min_n_letters, "params" : 3},
]

In [ ]:
from Image_Generator_helpers import  DataSets, set_paths, global_path, Random_Item
from data_filters import min_n_letters_raw
positions = ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10", "P11", "P12", "P13", "P14", "P15", "P16", "P17", "P18", "P19", "P20"]
set_obj = DataSets(set_paths, global_path, [min_n_letters_raw(3)], positions)
set_obj.csv_files


def get_velocity_labels(random_items):
    return [(item.csv_row["Tempo"].values.astype(np.float64)[0] / item.csv_row["Tempo Diff"].values.astype(np.float64)[0]) / 100 for item in random_items]

def position_labels_post_process(labels, set_obj: DataSets, random_items: list[Random_Item]):
    return np.array(labels)

In [ ]:
class Velocity_Image_Generator:
    def __init__(self, image_amount, image_preprocessors, noise_range):
        self.generator = Image_Generator_RAW(
            image_amount=image_amount,
            set_obj= set_obj,
            FFT_JUMP=64,
            batch_size=BATCH_SIZE,
            image_target_size=IMAGE_TARGET_SIZE,
            image_prepocessors=image_preprocessors,
            noise_range=noise_range,
            random_signal_indent=ADD_SIGNAL_INDENT_RANDOMLY,
            label_func = get_velocity_labels,
            label_post_process = position_labels_post_process,
            signal_cut_off=12860 * 6,
            position_probability=[0,2]
        )

Train_Generator_Raw = Velocity_Image_Generator(
    image_amount=BATCH_SIZE * EPOCH_SIZE,
    image_preprocessors=[],
    noise_range=ADD_NOISE_RANDOMLY
).generator
    

In [ ]:
# (t,l) = Train_Generator_Raw.__getitem__(0)


# for idx,img in enumerate(t):
#     print("label: ")
#     print (img.shape)
#     print(l[idx])
#     show_image(img, 1400)

In [ ]:
def conv_model_velocity(input_layer):
    x = keras.layers.Cropping2D(cropping=((0, 0), (0,200)), data_format=None)(input_layer)

    x = Conv2D(90,(1,7), padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(2,1),padding="same")(x)

    x = Conv2D(90,(1,7),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(1,5),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Flatten()(x)

    return x

In [ ]:
from tensorflow.keras import layers

input_layer = Input(shape=IMAGE_TARGET_SIZE)
conv_model_position_flattened = conv_model_velocity(input_layer)
output_layer_position = Dense(1, name="regr")(conv_model_position_flattened)

model = Model(inputs=input_layer, outputs=output_layer_position)
model.compile(loss=["mse"], optimizer='adam', metrics=["mean_absolute_error"])

print(model.summary())

In [ ]:
init_epoch = 0

In [ ]:

num_epochs = 2

def fit_model(epochs):
	
	global init_epoch
	history = model.fit(
		Train_Generator_Raw,
		steps_per_epoch = EPOCH_SIZE,
		epochs = epochs + init_epoch,
		initial_epoch=init_epoch,
		verbose =1,
		# validation_data = Validation_Generator_RAW,
		# validation_steps = 1000,
		workers=12,
		use_multiprocessing=True
	)
					
	init_epoch += epochs
	return history

history = fit_model(num_epochs)


In [ ]:
%%capture cap --no-stderr
from log_helpers.training_log import Training_Data_Log, print_name, json_to_file, get_deviating_predictions
import inspect

data_log = Training_Data_Log()
data_log.model_config = model.to_json()
data_log.model_config_method_string = [inspect.getsource(conv_model_velocity)]
data_log.noise_added = ADD_NOISE_RANDOMLY
data_log.training_data_masks = print_name(MASKS)
data_log.model_summary = model.summary()
data_log.model_optimizer = str(type(model.optimizer))
data_log.model_history = history.history
data_log.model_history_final_epoch = {k: v[-1] for k, v in history.history.items()}
data_log.total_epochs = init_epoch


In [ ]:
class Velocity_Results:
    total_predictions = None
    noise_level = None
    difference = None
    image_preprocessors_test = None
    predictions_off_by_more_than_difference = None
    predictions_incorrect_prercent = None
    model_evaluation = None

noise_levels = [[-2, 20], [0, 20]]
result_array = []
tempo_diff_above = 1

BATCHES_TESTING = 50

def velocity_comparer(prediction, correct):
    prediction = prediction * 100
    correct = correct * 100
    return abs(prediction[0] - correct) < tempo_diff_above

for noise_level in noise_levels:

    test_batch_generator = Velocity_Image_Generator(
        image_amount=BATCH_SIZE * BATCHES_TESTING,
        image_preprocessors=[],
        noise_range=noise_level
    ).generator

    velocity_differences  = get_deviating_predictions(test_batch_generator, model, velocity_comparer)
    evaluations = model.evaluate(test_batch_generator, verbose = 0)

    results = Velocity_Results()
    results.total_predictions = BATCHES_TESTING * BATCH_SIZE
    results.noise_level = noise_level
    results.difference = tempo_diff_above
    results.predictions_off_by_more_than_difference = len(velocity_differences)
    results.predictions_incorrect_prercent =  len(velocity_differences) / (BATCHES_TESTING * BATCH_SIZE) * 100
    results.model_evaluation = evaluations

    result_array.append(results.__dict__)


In [ ]:
data_log.results = result_array
data_log.results

In [ ]:
json_to_file("logs/velocity/velocity_data_log", data_log)

In [ ]:
for idx, diff in enumerate(velocity_differences):

    if idx > 10:
        break

    pred, correct, img = diff
    pred = pred * 100
    correct = correct * 100

    print('Prediction', round(pred[0]))
    print('Correct', round(correct))
    show_image(img, 500)

    print("----------------------------------------------------------------------------------------")
